# 1 - Calculate melting behaviour and phase proportions
These notebooks reproduce the results presented in Harðardottir et al. (manuscript submitted). Please contact Simon Matthews (simonm@hi.is) with any queries about the notebook.

The first step in the calculations is calculating the evolution of temperature and melt fraction during decompression melting of lithologically heterogeneous mantle. This is achieved using the pyMelt module. The pMELTS model is then run for each lithology, obtaining the previously calculated melt fraction at each step.

These calculations are very time consuming, and may timeout when using myBinder. However, the results are already provided, and so running this notebook may be skipped. The calculations may also be run on the ENKI server (see http://enki-portal.org/ for more info). If running on the ENKI server, the contents of this repository must be copied there.

To run the code, each cell must be run in turn. To run a cell, click inside of it, then hold down shift while you press enter. While the code is running a * will appear next to the cell. Once the calculation has completed, this will be replaced with a number.

First, import the required python modules:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyMelt as m
from thermoengine import equilibrate, core, phases, model
from scipy.optimize import root_scalar
from scipy.interpolate import interp1d
from copy import deepcopy

## Define functions for running pMELTS
First we will limit the calculation to phases we expect to be present during mantle melting:

In [ ]:
phase_inclusion = {'Actinolite':False, 'Aegirine':False, \
                   'Aenigmatite':False, 'Akermanite':False, 'Andalusite':False, \
                   'Anthophyllite':False, 'Apatite':False, 'Biotite':False, 'Chromite':False, \
                   'Coesite':False, 'Corundum':False, 'Cristobalite':False, 'Cummingtonite':False, \
                   'Fayalite':False, 'Forsterite':False, 'Gehlenite':False, 'Hematite':False, \
                   'Hornblende':False, 'Ilmenite':False, 'Ilmenite ss':False, 'Kalsilite':False, \
                   'Kalsilite ss':False, 'Kyanite':False, 'Leucite':False, 'Lime':False, \
                   'Liquid Alloy':False, 'Magnetite':False, 'Melilite':False, 'Muscovite':False, \
                   'Nepheline':False, 'Nepheline ss':False, 'OrthoOxide':False, 'Panunzite':False, \
                   'Periclase':False, 'Perovskite':False, 'Phlogopite':False, 'Quartz':False, \
                   'Rutile':False, 'Sanidine':False, 'Sillimanite':False, 'Solid Alloy':False, \
                   'Sphene':False, 'Tridymite':False, 'Whitlockite':False}

This function takes in the output from pyMelt, then runs pMELTS at each pressure step, whilst adjusting the bulk composition in accordance with near-fractional melting. The residual liquid porosity is set by `phi`, the value used in the calculations in the paper is 0.5%.

In [1]:
def fractionalmelt(column,phi=0.005):
    # Create dataframes to store the results
    res_kr4003 = pd.DataFrame(columns=['SiO2','TiO2','Al2O3','Fe2O3','FeO','MgO','CaO','Na2O','T_pyMelt','T_MELTS','F_MELTS','F_pyMelt','P'])
    res_kg1 = pd.DataFrame(columns=['SiO2','TiO2','Al2O3','Fe2O3','FeO','MgO','CaO','Na2O','T_pyMelt','T_MELTS','F_MELTS','F_pyMelt','P'])
    
    # Create a new instance of MELTS
    melts = equilibrate.MELTSmodel(version="5.6.1")
    melts.set_phase_inclusion_status(phase_inclusion)
    frac_coefs = melts.get_dictionary_of_default_fractionation_coefficients()
    
    # Set the starting bulk composition for each lithology:
    kr4003 = {'SiO2': 44.9,
              'TiO2':  0.16,
              'Al2O3': 4.26,
              'FeO':   7.85,
              'Fe2O3': 0.19,
              'MgO':  37.3,
              'CaO':   3.45,
              'Na2O':  0.22,
              'K2O':   0.0,
              'Cr2O3': 0.0,
              'P2O5':  0.0,
              'MnO':   0.0,
              'NiO':   0.0,
              'CoO':   0.0
             }

    kg1 = {'SiO2': 46.97,
           'TiO2':  0.78,
           'Al2O3': 9.75, 
           'FeO':   9.56,
           'Fe2O3': 0.23,
           'MgO':  23.57,
           'CaO':   7.35,
           'Na2O':  1.52,
           'K2O':   0.0,
           'Cr2O3': 0.0,
           'P2O5':  0.0,
           'MnO':   0.0,
           'NiO':   0.0,
           'CoO':   0.0
            }
    
    kr4003_mass = pd.Series(kr4003).sum()
    kg1_mass = pd.Series(kg1).sum()
    
    # Set fractionation coefficients such that only melt is removed from the system
    for ph in frac_coefs.keys():
        if ph != 'Liquid':
            frac_coefs[ph] = 0.0

    bulkcomp = kr4003
    bulkcompkg1 = kg1
    
    # Iterate through the pressure steps:
    for i in range(column.P.shape[0]):
        print('Step ' + str(i))
        p = np.round(column.P[i]*1000.0,0)
        flz = column.F['lz'][i]
        fpx = column.F['px'][i]
        t0 = np.round(column.Temperature[i],0)

        # Calculate the lherzolite melt composition, if it is melting
        if flz > 0:
            # Find the temperature at which the pyMelt melt fraction is obtained:
            t, melts, xmlout = findTfromF(bulkcomp, p, flz, t0, fractional_melting = True, initial_mass=kr4003_mass)
            
            # If the MELTS run has found melt all the way to zero as a result of a numerical error/bug, interpolate what the correct value should be:
            if t == False:
                t = res_kr4003['T_MELTS'][len(res_kr4003)-1] - (res_kr4003['T_MELTS'][len(res_kr4003)-2]-res_kr4003['T_MELTS'][len(res_kr4003)-1])
                x, melts, xmlout = calcF(t, p, bulkcomp)
                
            resdict = melts.get_composition_of_phase(xmlout,'Liquid')
            resdict['T_MELTS'] = t
            resdict['T_pyMelt'] = column.Temperature[i]
            resdict['X'] = melts.get_mass_of_phase(xmlout,'Liquid')/melts.get_mass_of_phase(xmlout)
            resdict['F_MELTS'] = (kr4003_mass - melts.get_mass_of_phase(xmlout))/kr4003_mass
            resdict['F_pyMelt'] = flz
            resdict['P'] = p
            
            for ph in melts.get_phase_names():
                mass = melts.get_mass_of_phase(xmlout,ph)
                if mass > 0:
                    resdict[ph] = mass
            
            # If the instantaneous melt fraction exceeds the porosity, fractionate the liquid.
            if resdict['X'] > phi:
                frac_to_remove = (resdict['X'] - phi)/resdict['X']
                frac_coefs['Liquid'] = frac_to_remove
                fractionated_comp = melts.fractionate_phases(xmlout,frac_coefs)
                bulkcomp_before = melts.get_composition_of_phase(xmlout)
                bulkcomp_after = {}
                # Adjust the bulk composition for the removal of melt
                for ox in fractionated_comp['Liquid'].keys():
                    bulkcomp_after[ox] = bulkcomp_before[ox] - fractionated_comp['Liquid'][ox]
                    if bulkcomp_after[ox] < 0:
                        bulkcomp_after[ox] = 0.0
                bulkcomp = bulkcomp_after
                for el in bulkcomp.keys():
                    bulkcomp[el] = np.round(bulkcomp[el],2)

            # Store the results from this step of the calculation:
            res_kr4003 = res_kr4003.append(resdict,ignore_index=True)



        if fpx > 0:
            # Find the temperature at which the pyMelt melt fraction is obtained:
            t, melts, xmlout = findTfromF(bulkcompkg1, p, fpx, t0, fractional_melting = True, initial_mass=kg1_mass)
            
            # If the MELTS run has found melt all the way to zero as a result of a numerical error/bug, interpolate what the correct value should be:
            if t == False:
                t = res_kg1['T_MELTS'][len(res_kg1)-1] - (res_kg1['T_MELTS'][len(res_kg1)-2]-res_kg1['T_MELTS'][len(res_kg1)-1])
                x, melts, xmlout = calcF(t, p, bulkcompkg1)
            
            resdict = melts.get_composition_of_phase(xmlout,'Liquid')
            resdict['T_MELTS'] = t
            resdict['T_pyMelt'] = column.Temperature[i]
            resdict['X'] = melts.get_mass_of_phase(xmlout,'Liquid')/melts.get_mass_of_phase(xmlout)
            resdict['F_MELTS'] = (kg1_mass - melts.get_mass_of_phase(xmlout))/kg1_mass
            resdict['F_pyMelt'] = fpx
            resdict['P'] = p

            for ph in melts.get_phase_names():
                mass = melts.get_mass_of_phase(xmlout,ph)
                if mass > 0:
                    resdict[ph] = mass
                    
            # If the instantaneous melt fraction exceeds the porosity, fractionate the liquid.
            if resdict['X'] > phi:
                frac_to_remove = (resdict['X'] - phi)/resdict['X']
                frac_coefs['Liquid'] = frac_to_remove
                fractionated_comp = melts.fractionate_phases(xmlout,frac_coefs)
                bulkcomp_before = melts.get_composition_of_phase(xmlout)
                bulkcomp_after = {}
                # Adjust the bulk composition for the removal of melt
                for ox in fractionated_comp['Liquid'].keys():
                    bulkcomp_after[ox] = bulkcomp_before[ox] - fractionated_comp['Liquid'][ox]
                    if bulkcomp_after[ox] < 0:
                        bulkcomp_after[ox] = 0.0
                bulkcompkg1 = bulkcomp_after
                for el in bulkcompkg1.keys():
                    bulkcompkg1[el] = np.round(bulkcompkg1[el],2)

            # Store the results from this step of the calculation:
            res_kg1 = res_kg1.append(resdict,ignore_index=True)
    
    return res_kr4003, res_kg1




At each step pMELTS should be run at the temperature required to reproduce the melt fraction calculated by pyMelt. However, there is no way of knowing what temperature this will be in advance. This function iteratively solves for the temperature. The precision can be adjusted by changing the arguments `course_dT` and `n_refinements`.

In [4]:
def findTfromF(bulk_comp, p, F, t0, course_dT=10.0, n_refinements=2, fractional_melting = False, initial_mass = 0.0):
    
    # First make sure we're starting below the solidus
    t = t0
 
    # Adjust the target melt fraction according to whether its for the bulk, or instantaneous melts.
    if fractional_melting == False:
        f = calcF(t, p, bulk_comp)[0]
    else:
        x, melts, xmlout = calcF(t, p, bulk_comp)
        mass_total = melts.get_mass_of_phase(xmlout)
        f = (initial_mass - mass_total)/initial_mass + x
        
    # Do the initial course search to bracket the temperature interval
    while f > F and t > 1000:
        t -= course_dT
        if fractional_melting == False:
            f, melts, xmlout = calcF(t, p, bulk_comp)
        else:
            x, melts, xmlout = calcF(t, p, bulk_comp)
            mass_total = melts.get_mass_of_phase(xmlout)
            f = (initial_mass - mass_total)/initial_mass + x
    
    # If MELTS has persistent melt, trigger a new function to deal with this on the return from this function.
    if t <= 1000:
        return False, False, False
    
    # Refine the search.
    for i in range(n_refinements+1):
        dT = course_dT/10.0**i
        while f < F:
            t += dT
            if fractional_melting == False:
                f, melts, xmlout = calcF(t, p, bulk_comp)
            else:
                x, melts, xmlout = calcF(t, p, bulk_comp)
                mass_total = melts.get_mass_of_phase(xmlout)
                f = (initial_mass - mass_total)/initial_mass + x
        if i != n_refinements:
            if fractional_melting == False:
                f, melts, xmlout = calcF(t, p, bulk_comp)
            else:
                x, melts, xmlout = calcF(t, p, bulk_comp)
                mass_total = melts.get_mass_of_phase(xmlout)
                f = (initial_mass - mass_total)/initial_mass + x
            t -= dT
    
    return t, melts, xmlout
    

Finally, this function does the work of actually running melts at a specified pressure, temperature, and bulk composition.

In [5]:
def calcF(t, p, bulk_comp):
    # Initialise a MELTS instance
    melts = equilibrate.MELTSmodel(version="5.6.1")
    melts.set_bulk_composition(bulk_comp)
    melts.set_phase_inclusion_status(phase_inclusion)
    # Run the MELTS calculation
    output = melts.equilibrate_tp(t,p, initialize=True)
    (status, t, p, xmlout) = output[0]
    # Extract the melt fraction
    f = melts.get_mass_of_phase(xmlout,'Liquid')/melts.get_mass_of_phase(xmlout)
    return f, melts, xmlout

## Run Calculations
In the paper we present results from three melting columns. Each column has a mantle potential temperature (specified in `Tps`), a pyroxenite fraction (`phi_pxs`), and a lithosphere thickness (`pliths`). The lithosphere can be made thicker in the following notebooks by truncating the results, so these need not be the final values.

In [13]:
Tps = [1451,1550,1549] # in degC
phi_pxs = [0.025,0.025,0.05]
Plith = [2.0,0.5,1.0] # in GPa

n = len(Tps)

The calculations are then run in the following cell. This will take some time. The code will print an update at each step so that it is clear whether the code has crashed. If it crashes, the kernel must be restarted, and all the cells run again. If one or more column was successfully calculated, the problematic column can be run alone by altering the line indicated below.

In [14]:
lz = m.LithologyKLB1()
px = m.LithologyKG1()

results = []

for i in range(n):
    if i > -1: # Change this line to run a specific column again.
        print(str(i+1) +' of '+str(n))
        # Define the pyMelt mantle object:
        mantle = m.mantle([lz,px],[1-phi_pxs[i],phi_pxs[i]],['lz','px'])
        # Run a pyMelt melting calculation:
        column = mantle.AdiabaticMelt_1D(np.round(Tps[i],0),Pend=np.round(Plith[i],3),steps=int(8.0-Plith[i])*50+1)
        # Use the result to run a pMELTS fractional melting calculation:
        res_kr4003, res_kg1 = fractionalmelt(column,phi=0.005)
        # Save the results as csv files.
        res_kr4003.to_csv('kr4003_gradient_'+str(i)+'.csv')
        res_kg1.to_csv('kg1_gradient_'+str(i)+'.csv')


1 of 3
Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Step 8
Step 9
Step 10
Step 11
Step 12
Step 13
Step 14
Step 15
Step 16
Step 17
Step 18
Step 19
Step 20
Step 21
Step 22
Step 23
Step 24
Step 25
Step 26
Step 27
Step 28
Step 29
Step 30
Step 31
Step 32
Step 33
Step 34
Step 35
Step 36
Step 37
Step 38
Step 39
Step 40
Step 41
Step 42
Step 43
Step 44
Step 45
Step 46
Step 47
Step 48
Step 49
Step 50
Step 51
Step 52
Step 53
Step 54
Step 55
Step 56
Step 57
Step 58
Step 59
Step 60
Step 61
Step 62
Step 63
Step 64
Step 65
Step 66
Step 67
Step 68
Step 69
Step 70
Step 71
Step 72
Step 73
Step 74
Step 75
Step 76
Step 77
Step 78
Step 79
Step 80
Step 81
Step 82
Step 83
Step 84
Step 85
Step 86
Step 87
Step 88
Step 89
Step 90
Step 91
Step 92
Step 93
Step 94
Step 95
Step 96
Step 97
Step 98
Step 99
Step 100
Step 101
Step 102
Step 103
Step 104
Step 105
Step 106
Step 107
Step 108
Step 109
Step 110
Step 111
Step 112
Step 113
Step 114
Step 115
Step 116
Step 117
Step 118
Step 119
Step 120
Step 121
Step 